In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import nibabel as nib
from nilearn.plotting import plot_design_matrix, plot_stat_map, view_img
from nilearn.reporting import get_clusters_table
from nilearn import datasets
import glob
import os
import numpy as np
import pandas as pd

import sys
sys.path.append('/Users/zeynepenkavi/Documents/RangelLab/DescribedVsLearned_fmri/analysis/03_level3')
from make_design_files import make_design_files

/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [ ]:
def filter_corrp_ts(tval_img, pval_img, threshold=0.95):
    if isinstance(tval_img, str):
        tval_img = nib.load(tval_img)
    if isinstance(pval_img, str):
        pval_img = nib.load(pval_img)
        
    tval_data = tval_img.get_fdata()
    pval_data = pval_img.get_fdata()
    filt_tval_data = np.where(pval_data > threshold, tval_data, 0)
    filt_tval_img = nib.Nifti1Image(filt_tval_data.astype(np.float64), tval_img.affine)
    
    return filt_tval_img

def get_filt_tval_img(reg, threshold=0.95):
    pval_fn = '%s/rand_model1_%s_tfce_corrp_tstat1.nii.gz'%(reg, reg)
    tval_fn = '%s/rand_model1_%s_tstat1_tfce.nii.gz'%(reg, reg)
    pval_img = os.path.join(level3_path, pval_fn)
    tval_img = os.path.join(level3_path, tval_fn)
    filt_tval_img = filter_corrp_ts(tval_img, pval_img, threshold=threshold)
    
    return filt_tval_img

def plot_filt_tval_img(reg, threshold=0.95, interactive=False):
    filt_tval_img = get_filt_tval_img(reg, threshold=threshold)
    if(len(np.unique(filt_tval_img.get_fdata())) == 1):
        print('Nothing survives correction for %s'%(reg))
    elif interactive:
        view = view_img(filt_tval_img, 
             draw_cross=False,
             title='%s'%(reg))
        
        return view
    else:
        print('Plotting tvalues filtered for corrected p values > 0.05')
        plot_stat_map(filt_tval_img, 
             draw_cross=False,
             title='%s'%(reg))

In [ ]:
level3_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/bids_nifti_wface/derivatives/nilearn/glm/level3/model1'

# Event regressors

In [ ]:
regs = ['fractalProb', 'stim', 'reward']
for reg in regs:
    plot_filt_tval_img(reg)

# Parametric regressors

In [ ]:
regs = ['fractalProbParam', 'valDiff', 'rewardParam', 'rpe']
for reg in regs:
    plot_filt_tval_img(reg)

In [ ]:
reg = 'rpe'
pval_pattern = '%s/rand_model1_%s_tfce_corrp_tstat1.nii.gz'%(reg, reg)
pval_img = os.path.join(level3_path, pval_pattern)
get_clusters_table(pval_img, stat_threshold=.95, cluster_threshold=20)

In [ ]:
view = plot_filt_tval_img(reg, 
                          interactive=True)
view

# Contrasts

In [ ]:
regs = ['conflict', 'noconflict']
for reg in regs:
    plot_filt_tval_img(reg)

In [ ]:
regs = ['conflict_vs_noconflict']
for reg in regs:
    plot_filt_tval_img(reg)

In [ ]:
tval_fn = '%s/rand_model1_%s_tstat1_tfce.nii.gz'%(reg, reg)
tval_img = os.path.join(level3_path, tval_fn)
plot_stat_map(tval_img, threshold=3, draw_cross=False)

# Interactions with p(Fractal)

# Slow vs fast learner differences

In [2]:
mnum = 'model2'
learner_info_path = '/Users/zeynepenkavi/Documents/RangelLab/DescribedVsLearned_fmri/analysis/03_level3/learner_info.csv'
make_design_files(mnum, learner_info_path)

***********************************************
Saving design matrix for model2
***********************************************
***********************************************
Saving contrast matrix for model2
***********************************************
